In [ ]:
import os
import pickle as pk
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from helper_utils.utils import *

In [ ]:
# Load data

data_directory = "processed_data"
data_filename_base = "X_top300_201801_201912_30_m_final"
data_filename_ext = ".npz"
data_filename = data_filename_base+data_filename_ext

data = {}
with np.load(os.path.join(data_directory, data_filename)) as npz_loader:
    for key in npz_loader.files:
        print("{}: {}".format(key, npz_loader[key].shape))
        data[key] = npz_loader[key]

X_train = data["X_train"]
y_train = data["y_train"]
X_val = data["X_val"]
y_val = data["y_val"]
X_test = data["X_test"]
y_test = data["y_test"]


input_shape = tuple(list(X_train.shape)[1:])
output_shape = tuple(list(y_train.shape)[1:])
input_dims = np.product(input_shape)
output_dims = np.product(output_shape)

num_nodes, num_steps, input_features = input_shape
_, _, output_features = output_shape

In [ ]:
from models.fnn import FNN
from models.linear import LinearRegressor
from models.fclstm import FCLSTM
from models.stgcn import STGCN
from models.wavenet import GraphWaveNet, GraphConv

model_types = {'FNN': FNN, 'LinearRegressor': LinearRegressor,
               'FCLSTM': FCLSTM, 'STGCN': STGCN, 'GraphWaveNet': GraphWaveNet}


In [ ]:
from specs_to_run import *

training_specs, model_params = models_to_run['STGCN_adaptiveA']

MODEL_CLASS = model_types[training_specs['model_type']]
BATCH_SIZE = training_specs['batch_size']
MAX_EPOCHS = training_specs['max_epochs']
PATIENCE = training_specs['patience']
LOSS_TYPE = training_specs['loss_type']
LEARNING_RATE = training_specs['learning_rate']

callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE)

In [ ]:
tf.__version__

In [ ]:
tf.random.set_seed(seed=10707)

model = MODEL_CLASS(model_params)

model.compile(loss=LOSS_TYPE, optimizer=tf.keras.optimizers.Adam(LEARNING_RATE))
print(model(X_train[:32]).shape)
model.summary()

In [ ]:
history = model.fit(x=X_train, y=y_train, shuffle=True,
                    batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                    validation_data=(X_val,y_val),
                    callbacks=[callback_es])

In [ ]:
# EVALUATION
plt.figure()
plt.xlabel("Epochs")
plt.ylabel(LOSS_TYPE)
plt.plot(history.history['loss'],label=training_specs['model_name']+'_train')
plt.plot(history.history['val_loss'],label=training_specs['model_name']+'_val')
plt.legend()

print(training_specs['model_name']+" test Loss = ", model.evaluate(X_test,y_test))

In [ ]:
predictions = model.predict(X_test)
ground_truth = y_test

In [ ]:
model_store_directory = "stored_models"
run_info_store_directory = "stored_models"

if training_specs['save_model']:
    model_store_filename = "_".join([training_specs['model_name'], "model"])
    model_store_filepath = os.path.join(model_store_directory, model_store_filename)
    model.save(model_store_filepath, save_format="tf")

if training_specs['save_model_history']:
    run_info = {}
    run_info["train_test_path"] = data_filename
    run_info["history"] = history.history
    run_info["predictions"] = predictions

    run_info_store_filename = "_".join([training_specs['model_name'], "run_info"]) + ".pk"
    run_info_store_filepath = os.path.join(run_info_store_directory, run_info_store_filename)
    pk.dump(run_info, open(run_info_store_filepath, "wb"))

In [ ]:
station = 50
horizon = 8
feature = 0
feature_labels = {0: 'Demand', 1: 'Supply'}

horizon_predictions = seq_from_windows(predictions, horizon=horizon)
horizon_ground_truth = seq_from_windows(y_test, horizon=horizon)

plt.figure(figsize=(20,5))
plt.xlim(6000,7000)
plt.title(f"Prediction horizon = {horizon * 30} min")
plt.xlabel("Time / 30 min")
plt.ylabel(f"{feature_labels[feature]} at station {station}")
plt.plot(horizon_predictions[station,:,feature],label=training_specs['model_name'])
plt.plot(horizon_ground_truth[station,:,feature],label='ground truth')
plt.legend()